In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
#from keras.optimizers import gradient_descent_v2 
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


479 documents
43 classes ['Director', 'admission', 'autohod', 'canteen', 'civilhod', 'college intake', 'comphod', 'course', 'creator', 'document', 'elechod', 'event', 'facilities', 'faculty', 'faculty portal', 'fees', 'fest', 'goodbye', 'greeting', 'hod', 'hostel', 'hours', 'infrastructure', 'ithod', 'library', 'location', 'mechhod', 'menu', 'name', 'number', 'pehod', 'phyhod', 'placement', 'ragging', 'salutaion', 'scholarship', 'sports', 'student portal', 'swear', 'syllabus', 'task', 'transport', 'uniform']
267 unique lemmatized words ["'s", 'a', 'about', 'active', 'address', 'admision', 'admission', 'allotment', 'am', 'an', 'and', 'antiragging', 'any', 'anyone', 'application', 'are', 'as', 'asshole', 'at', 'attend', 'auto', 'automobile', 'available', 'average', 'be', 'between', 'big', 'bitch', 'book', 'branch', 'bring', 'building', 'bulding', 'bus', 'by', 'bye', 'ca', 'cafetaria', 'call', 'called', 'can', 'canteen', 'capacity', 'case', 'casuals', 'chatting', 'civil', 'code', 'college

C:\Users\HP\AppData\Local\Temp/ipykernel_5420/1893352581.py:74: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


96/96 [==============================] - 1s 3ms/step - loss: 3.6918 - accuracy: 0.0626
Epoch 2/200
96/96 [==============================] - 0s 3ms/step - loss: 3.4707 - accuracy: 0.1127
Epoch 3/200
96/96 [==============================] - 0s 3ms/step - loss: 3.1820 - accuracy: 0.1816
Epoch 4/200
96/96 [==============================] - 0s 3ms/step - loss: 2.8511 - accuracy: 0.2797
Epoch 5/200
96/96 [==============================] - 0s 3ms/step - loss: 2.5403 - accuracy: 0.3340
Epoch 6/200
96/96 [==============================] - 0s 3ms/step - loss: 2.3642 - accuracy: 0.3820
Epoch 7/200
96/96 [==============================] - 0s 3ms/step - loss: 2.0675 - accuracy: 0.4468
Epoch 8/200
96/96 [==============================] - 0s 3ms/step - loss: 1.8485 - accuracy: 0.5010
Epoch 9/200
96/96 [==============================] - 0s 3ms/step - loss: 1.6617 - accuracy: 0.5407
Epoch 10/200
96/96 [==============================] - 0s 3ms/step - loss: 1.4502 - accuracy: 0.5866
Epoch 11/200
96/96 [=

96/96 [==============================] - 0s 3ms/step - loss: 0.1721 - accuracy: 0.9499
Epoch 84/200
96/96 [==============================] - 0s 3ms/step - loss: 0.1697 - accuracy: 0.9395
Epoch 85/200
96/96 [==============================] - 0s 4ms/step - loss: 0.2160 - accuracy: 0.9269
Epoch 86/200
96/96 [==============================] - 0s 3ms/step - loss: 0.1796 - accuracy: 0.9415
Epoch 87/200
96/96 [==============================] - 0s 3ms/step - loss: 0.1420 - accuracy: 0.9624
Epoch 88/200
96/96 [==============================] - 0s 3ms/step - loss: 0.1498 - accuracy: 0.9499
Epoch 89/200
96/96 [==============================] - 0s 3ms/step - loss: 0.1768 - accuracy: 0.9415
Epoch 90/200
96/96 [==============================] - 0s 3ms/step - loss: 0.1679 - accuracy: 0.9436
Epoch 91/200
96/96 [==============================] - 0s 3ms/step - loss: 0.2113 - accuracy: 0.9436
Epoch 92/200
96/96 [==============================] - 0s 3ms/step - loss: 0.1916 - accuracy: 0.9374
Epoch 93/200


96/96 [==============================] - 0s 3ms/step - loss: 0.1278 - accuracy: 0.9582
Epoch 165/200
96/96 [==============================] - 0s 3ms/step - loss: 0.0794 - accuracy: 0.9708
Epoch 166/200
96/96 [==============================] - 0s 4ms/step - loss: 0.1583 - accuracy: 0.9541
Epoch 167/200
96/96 [==============================] - 0s 3ms/step - loss: 0.1128 - accuracy: 0.9624
Epoch 168/200
96/96 [==============================] - 0s 3ms/step - loss: 0.0999 - accuracy: 0.9687
Epoch 169/200
96/96 [==============================] - 0s 3ms/step - loss: 0.1196 - accuracy: 0.9582
Epoch 170/200
96/96 [==============================] - 0s 3ms/step - loss: 0.0897 - accuracy: 0.9687
Epoch 171/200
96/96 [==============================] - 0s 3ms/step - loss: 0.1180 - accuracy: 0.9645
Epoch 172/200
96/96 [==============================] - 0s 3ms/step - loss: 0.0999 - accuracy: 0.9749
Epoch 173/200
96/96 [==============================] - 0s 3ms/step - loss: 0.1444 - accuracy: 0.9541
Epoc

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


#Creating GUI with tkinter
import tkinter
from tkinter import *
from PIL import Image,ImageTk


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = tkinter.Toplevel()
base.title("Edubot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
#base.photoimg(img_chat)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

img_chat=Image.open('chat.jpg')
img_chat=img_chat.resize((150,70),Image.ANTIALIAS)
base.photoimg=ImageTk.PhotoImage(img_chat)
Title_label = Label(ChatLog,bd=0,relief=RAISED,anchor='nw',width=730,compound=LEFT,image=base.photoimg,text='Edubot', font=('arial',30,'bold'),fg='green',bg='white')
Title_label.pack(side=TOP)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=2,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
#ClearButton.place(x=350,y=401,height=90)

base.mainloop()


1/1 [==============================] - 0s 19ms/step


In [ ]:
pip install tensorflow